# Импорт и функции

In [1]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

import gradio as gr

In [2]:
def columns_drop(train_full_info):
    
    mask_avg_dep = train_full_info.columns.str.startswith('avg_dep_avg_balance_fact_')
    mask_zp = train_full_info.columns.str.startswith('zp_')
    mask_agg = (train_full_info.columns.str.startswith(('max_', 'min_', 'avg_', 'sum_'))
                & ~train_full_info.columns.isin(['min_max_dep_balance_amt']))
    mask_dep = train_full_info.columns.str.startswith('dep_')
    mask_income = train_full_info.columns.str.startswith('income_')
    mask_cnt = train_full_info.columns.str.startswith('cnt_')

    cols_to_delete = []
    cols_to_delete = cols_to_delete + train_full_info.columns[mask_avg_dep].tolist()
    cols_to_delete = cols_to_delete + train_full_info.columns[mask_zp].tolist()
    cols_to_delete = cols_to_delete + train_full_info.columns[mask_agg].tolist()
    cols_to_delete = cols_to_delete + train_full_info.columns[mask_dep].tolist()
    cols_to_delete = cols_to_delete + train_full_info.columns[mask_income].tolist()
    cols_to_delete = cols_to_delete + train_full_info.columns[mask_cnt].tolist()

    cols_to_delete = cols_to_delete + ['cnt_prolong_max_5y', 'app_real_estate_ind', 
     'app_children_cnt', 'app_dependent_cnt', 'app_family_cnt',
     'vehicle_counrty_type_nm', 'used_car_flg', 'app_vehicle_ind',
     'savings_pension_flg', 'savings_broker_flg',
     'customer_age', 'savings_sum_bro_now', 'app_income_app']

    train_full_info = train_full_info.drop(cols_to_delete, axis=1)
    
    return train_full_info

In [3]:
from collections import defaultdict

def filter_by_variance(df, threshold=0.01, exclude_columns=None):
    """
    Удаляет колонки с дисперсией ниже заданного порога.
    
    Параметры:
    - df: исходный DataFrame
    - threshold: порог дисперсии (по умолчанию 0.01)
    - exclude_columns: список колонок, которые не нужно удалять (например, целевая переменная)
    
    Возвращает:
    - DataFrame без колонок с низкой дисперсией
    """
    if exclude_columns is None:
        exclude_columns = []
    
    # Выбираем только числовые колонки, исключая указанные
    numeric_cols = df.select_dtypes(include=['number']).columns.difference(exclude_columns)
    
    # Вычисляем дисперсию
    variances = df[numeric_cols].var()
    
    # Колонки для удаления
    low_variance_cols = variances[variances < threshold].index.tolist()
    
    # Удаляем колонки с низкой дисперсией
    return df.drop(columns=low_variance_cols)

def filter_by_low_correlation(df, target_col=['id', 'index', 'target'], threshold=0.01):
    """
    Оставляет только колонки с корреляцией (по модулю) выше порога с целевой переменной.
    
    Параметры:
    - df: исходный DataFrame
    - target_col: название целевой колонки
    - threshold: порог корреляции (по умолчанию 0.4)
    
    Возвращает:
    - DataFrame с колонками, имеющими значимую корреляцию с target_col
    """
    
    # Вычисляем корреляцию только для числовых колонок
    numeric_df = df.select_dtypes(include=['number'])
    corr_matrix = numeric_df.corr()
    
    # Получаем корреляцию с целевой переменной
    target_corr = corr_matrix.loc[target_col, :]
    
    # Выбираем колонки с корреляцией выше порога (по модулю)
    significant_cols = target_corr.columns[(target_corr.abs() > threshold).any(axis=0)]

    
    # Оставляем только значимые колонки
    return df[significant_cols]

def remove_temporal_duplicates(df, target_col=None):
    """
    Удаляет временные дубликаты колонок с особым правилом для _now, _1m, _12m:
    - Всегда оставляет _now, _1m и _12m, если они существуют
    - Для остальных временных периодов применяет стандартные правила приоритета
    
    Параметры:
    - df: исходный DataFrame
    - target_col: название колонки, которую нужно сохранить
    
    Возвращает:
    - DataFrame с сохраненными временными колонками
    """
    from collections import defaultdict
    import re
    
    col_groups = defaultdict(list)
    temporal_pattern = re.compile(r'(_now|_\d{1,2}m|_\d{1,2}|_7d)$')
    
    # Группируем колонки по базовым названиям
    for col in df.columns:
        if target_col and col == target_col:
            continue  # Целевую колонку не трогаем
            
        # Ищем временной суффикс
        match = temporal_pattern.search(col)
        if match:
            suffix = match.group()
            base_col = col[:-len(suffix)]
            col_groups[base_col].append((suffix, col))
        else:
            # Колонки без временного суффикса оставляем как есть
            col_groups[col].append(('', col))
    
    cols_to_keep = set()
    cols_to_remove = set()
    
    # Определяем приоритетные суффиксы (которые оставляем все)
    keep_all_suffixes = ['_now', '_1m', '_12m']
    # Стандартные приоритеты для остальных случаев
    standard_priority = ['_now', '_1m', '_1', '_12m', '_12', '_2m', '_2', 
                        '_3m', '_3', '_6m', '_6', '_9m', '_9', '_7d']
    
    for base_col, variants in col_groups.items():
        if len(variants) > 1:  # Если есть временные варианты
            # Проверяем наличие специальных суффиксов
            special_variants = [(suf, col) for suf, col in variants if suf in keep_all_suffixes]
            
            if len(special_variants) >= 1:  # Если есть хотя бы один специальный суффикс
                # Оставляем все специальные суффиксы
                for suf, col in special_variants:
                    cols_to_keep.add(col)
                
                # Удаляем остальные временные варианты для этой группы
                for suf, col in variants:
                    if suf not in keep_all_suffixes:
                        cols_to_remove.add(col)
            else:
                # Применяем стандартные правила приоритета
                sorted_variants = sorted(
                    variants,
                    key=lambda x: standard_priority.index(x[0]) if x[0] in standard_priority 
                    else len(standard_priority)
                )
                # Оставляем колонку с наивысшим приоритетом
                cols_to_keep.add(sorted_variants[0][1])
                # Остальные добавляем на удаление
                for suf, col in sorted_variants[1:]:
                    cols_to_remove.add(col)
    
    # Удаляем дубликаты
    result_df = df.drop(columns=cols_to_remove)
    
    return result_df


def preprocess_data(df, mode='both', nan_threshold=0.7):
    """
    Удаляет колонки с долей пропусков (NaN) выше заданного порога.
    
    Параметры:
        df: Исходный DataFrame.
        mode: Режим обработки:
            - 'both' (по умолчанию): все колонки.
            - 'numeric': только числовые колонки.
            - 'categorical': только категориальные колонки.
        nan_threshold: Доля пропусков для удаления колонки (0.7 = 70%).
    
    Возвращает:
        DataFrame с удаленными колонками.
    """
    # Определяем типы колонок
    numeric_cols = df.select_dtypes(include=['int', 'float']).columns
    categorical_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns
    
    # Выбираем колонки для обработки в зависимости от режима
    if mode == 'numeric':
        cols_to_check = numeric_cols
    elif mode == 'categorical':
        cols_to_check = categorical_cols
    else:  # 'both'
        cols_to_check = df.columns
    
    # Вычисляем порог для удаления
    threshold = len(df) * nan_threshold
    
    # Удаляем колонки с пропусками > порога
    cols_to_drop = [col for col in cols_to_check if df[col].isnull().sum() > threshold]
    df = df.drop(columns=cols_to_drop)
    
    return df


def remove_correlated_features(df, target_col='target', threshold=0.5):
    """
    Удаляет коррелирующие признаки, сохраняя целевую переменную.
    
    Параметры:
        df: Исходный датафрейм.
        target_col: Название столбца с целевой переменной (не удаляется).
        threshold: Порог корреляции для удаления (по умолчанию 0.5).
    
    Возвращает:
        Датафрейм с некоррелирующими признаками и целевой переменной.
    """
    # Отделяем целевую переменную и признаки
    target = df[target_col]
    features = df[df.select_dtypes(include=['number']).columns]
    features = features.drop(columns=[target_col])
    
    # Вычисляем матрицу корреляций
    corr_matrix = features.corr().abs()
    
    # Верхний треугольник матрицы (чтобы не дублировать пары)
    upper = corr_matrix.where(np.triu(np.ones_like(corr_matrix, dtype=bool), k=1))
    
    # Находим признаки для удаления (корреляция > threshold)
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    # Удаляем коррелирующие признаки и возвращаем целевой столбец
    filtered_features = features.drop(columns=to_drop)
    return pd.concat([filtered_features, target], axis=1)

# Обработка данных и модель

In [4]:
train_main_df = pd.read_parquet('input_data/train_main_df.parquet')
train_target = pd.read_csv('input_data/train_target.csv')

train_target['index'] = train_target.index
train_main_df['index'] = train_main_df.index

train = pd.merge(train_target, train_main_df, on=['id', 'index'])

In [5]:
train = columns_drop(train)
train = filter_by_variance(train)
train = remove_temporal_duplicates(train)
train = remove_correlated_features(train)
train = filter_by_low_correlation(train, threshold=0.01)

In [6]:
train.columns

Index(['id', 'index', 'min_max_dep_balance_amt', 'savings_sum_dep_now',
       'savings_sum_dep_12m', 'savings_sum_sa_now', 'savings_sum_sa_12m',
       'savings_sum_sa_credit_1m', 'savings_sum_sa_credit_12m',
       'savings_sum_bro_1m', 'target'],
      dtype='object')

In [7]:
train

,id,index,min_max_dep_balance_amt,savings_sum_dep_now,savings_sum_dep_12m,savings_sum_sa_now,savings_sum_sa_12m,savings_sum_sa_credit_1m,savings_sum_sa_credit_12m,savings_sum_bro_1m,target
0,97678374,0,NaN,0.000000,0.000000,2.645298e+02,0.000000,0.000000,0.000000e+00,219.018234,0.000000
1,62472650,1,NaN,4963.610840,0.000000,2.039230e+02,289.098999,0.000000,2.163016e+02,0.000000,0.000000
2,94308112,2,130510.773438,0.000000,2376.036621,2.211489e+05,126578.445312,1472.557983,1.858215e+05,54.809654,219932.906250
3,68994873,3,57.079170,8969.149414,334.324036,1.529677e+03,8470.016602,166.532135,1.240958e+05,171.854828,631.770020
4,78127603,4,NaN,4500.253906,0.000000,3.650610e+02,207.131500,183.979782,0.000000e+00,174.200806,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
85584,6163462,85584,819.466614,3354.238037,0.000000,0.000000e+00,0.000000,290.803802,2.232204e+03,81.305336,0.000000
85585,22320709,85585,1243.621704,6023.775391,523.401062,1.062631e+06,119322.023438,236934.421875,2.299806e+06,227.499557,841071.000000
85586,23863045,85586,0.010000,0.000000,2228.442383,0.000000e+00,0.000000,150.719467,0.000000e+00,9.497079,27.990000
85587,33408678,85587,0.010000,0.000000,461.857391,2.027054e+05,0.000000,202529.312500,2.039604e+05,0.000000,0.000000


In [8]:
# min_max_dep_balance_amt - Минимальный остаток по всем счетам клиента за последний год (сокращен по причине высокой нагрузки)
# savings_sum_dep_now - Суммарный остаток по продукту Deposit на отчетную дату
# savings_sum_dep_12m - Суммарный остаток по продукту Deposit на отчетную дату минус 12 месяцев
# savings_sum_sa_now - Суммарный остаток по продукту Save_account  на отчетную дату
# savings_sum_sa_12m - Суммарный остаток по продукту Save_account  на отчетную дату минус 12 месяцев
# savings_sum_sa_credit_1m - Сумма поступлений денежных средств на счета с продуктом Save_account  за 1 месяц до отчетной даты
# savings_sum_sa_credit_12m - Сумма поступлений денежных средств на счета с продуктом Save_account  за 12 последних месяцев до отчетной даты
# savings_sum_bro_1m - Суммарный остаток по продукту инвест. (брокерские счета + ИИС)  на отчетную дату минус 1 месяц

In [10]:
features = train.columns
categorical_features = train[features].select_dtypes(include=['object']).columns
for feature in categorical_features:
    train[feature] = train[feature].astype(str)
categorical_features_indices = np.where(train.dtypes == 'object')[0]

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    train.drop(['id', 'target', 'index'], axis=1),
    train_target['target'], 
    test_size=0.7,
    random_state=42 
)

y_train = np.log1p(y_train)
y_test = np.log1p(y_test)

train_pool = Pool(data = X_train, 
                  label = y_train, 
                  cat_features = categorical_features_indices)

test_pool = Pool(data = X_test, 
                  cat_features = categorical_features_indices)

model = CatBoostRegressor(
    bootstrap_type='MVS',
    grow_policy='Lossguide',
    max_leaves=32,
    min_data_in_leaf=30,
    boosting_type='Plain',
    score_function='L2',
    iterations=300,
    learning_rate=0.05,
    loss_function='RMSE',
    random_seed=42,
    verbose = 1,
    cat_features=categorical_features_indices
)

model.fit(train_pool)

0:	learn: 5.3945605	total: 178ms	remaining: 53.3s
1:	learn: 5.1819814	total: 191ms	remaining: 28.5s
2:	learn: 4.9817551	total: 204ms	remaining: 20.2s
3:	learn: 4.7932667	total: 214ms	remaining: 15.8s
4:	learn: 4.6162079	total: 220ms	remaining: 13s
5:	learn: 4.4502712	total: 227ms	remaining: 11.1s
6:	learn: 4.2943229	total: 234ms	remaining: 9.79s
7:	learn: 4.1486550	total: 241ms	remaining: 8.79s
8:	learn: 4.0126945	total: 248ms	remaining: 8.01s
9:	learn: 3.8849895	total: 255ms	remaining: 7.38s
10:	learn: 3.7665640	total: 262ms	remaining: 6.87s
11:	learn: 3.6559510	total: 268ms	remaining: 6.44s
12:	learn: 3.5521785	total: 275ms	remaining: 6.07s
13:	learn: 3.4555944	total: 282ms	remaining: 5.75s
14:	learn: 3.3659702	total: 288ms	remaining: 5.48s
15:	learn: 3.2826986	total: 295ms	remaining: 5.24s
16:	learn: 3.2054684	total: 302ms	remaining: 5.03s
17:	learn: 3.1335985	total: 309ms	remaining: 4.84s
18:	learn: 3.0670861	total: 316ms	remaining: 4.67s
19:	learn: 3.0056579	total: 323ms	remaining

In [11]:
# Вычисляем RMSE
rmse = np.sqrt(np.mean((y_test - model.predict(test_pool)) ** 2))
print(f"RMSE: {rmse:.4f}")

# codabench score 2.48

RMSE: 2.3537


# Инференс

In [21]:
def predict_balance(min_max_dep_balance_amt, savings_sum_dep_now,
                    savings_sum_dep_12m, savings_sum_sa_now, savings_sum_sa_12m, savings_sum_sa_credit_1m,
                    savings_sum_sa_credit_12m, savings_sum_bro_1m):
    
    input_data = pd.DataFrame([[min_max_dep_balance_amt, savings_sum_dep_now,
                                savings_sum_dep_12m, savings_sum_sa_now, savings_sum_sa_12m, savings_sum_sa_credit_1m,
                                savings_sum_sa_credit_12m, savings_sum_bro_1m]],
                              columns=train.drop(['id', 'target', 'index'], axis=1).columns
    )

    prediction = model.predict(input_data)

    return f"Предполагаемый остаток на счете: {(np.exp(prediction) - 1)[0]}"

In [23]:
inputs = [
    gr.Slider(0, 500_000, step=1000, value=5000, 
              label="Минимальный остаток по счетам за год (руб)", 
              info="min_max_dep_balance_amt"),
    gr.Slider(0, 2_000_000, step=10000, value=300_000, 
              label="Остаток по депозитам на текущую дату", 
              info="savings_sum_dep_now"),
    gr.Slider(0, 2_000_000, step=10000, value=200_000, 
              label="Остаток по депозитам 12 мес назад (руб)", 
              info="savings_sum_dep_12m"),
    gr.Slider(0, 2_000_000, step=10000, value=300_000, 
              label="Текущий остаток по накопительным счетам (руб)", 
              info="savings_sum_sa_now"),
    gr.Slider(0, 1_000_000, step=10000, value=150_000, 
              label="Остаток по накопительным счетам 12 мес назад (руб)", 
              info="savings_sum_sa_12m"), 
    gr.Slider(0, 500_000, step=5000, value=50_000, 
              label="Поступления на накопительные счета за 1 мес (руб)", 
              info="savings_sum_sa_credit_1m"),
    gr.Slider(0, 2_000_000, step=10000, value=600_000, 
              label="Поступления на накопительные счета за 12 мес (руб)", 
              info="savings_sum_sa_credit_12m"),
    gr.Slider(0, 5_000_000, step=10000, value=1_200_000, 
              label="Суммарный остаток по продукту инвестиций 1 месяц назад", 
              info="savings_sum_bro_1m")
]

title = "Предсказательная модель остатка на счете"
description = "Введите финансовые показатели клиента для анализа и прогнозирования. Все суммы указываются в рублях."

demo = gr.Interface(
    fn=predict_balance,
    inputs=inputs,
    outputs="text",
    title=title,
    description=description
)

demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
